# Группировка и агрегация данных
Теперь научимся группировать данные.

In [21]:
import pandas as pd
students_performance = pd.read_csv("../../../Datasets/StudentsPerformance.csv")
students_performance.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


Решим такую задачку: посчитаем среднее значение score'ов в зависимости от пола студентов.
<br>Docs:https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html

In [22]:
students_performance.groupby("gender").mean(numeric_only=True)
# Параметр numeric_only=True нужен чисто для того, чтобы избавится от предупреждения.
# Подробнее об этом тут: https://stackoverflow.com/questions/72223610/dropping-invalid-columns-futurewarning

,math score,reading score,writing score
gender,,,
female,63.633205,72.608108,72.467181
male,68.728216,65.473029,63.311203


Достаточно просто, да? Однако если бы мы хотели применить несколько функций к нашим сгруппированным данным, то такой подход был бы слегка затруднителен. 

Мы можем более подробно расписать, как именно мы хотим сделать группировку. Воспользуемся методом **.aggregate()**.
<br>Docs: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.aggregate.html

In [23]:
students_performance.groupby("gender").aggregate({
    "math score": "mean",
    "reading score": "mean"
})

,math score,reading score
gender,,
female,63.633205,72.608108
male,68.728216,65.473029


Как мы видим, мы получили датафрейм, в котором в качестве индексов строк выступают названия переменных, по которым мы проводим агрегацию. Это не всегда желаемое поведение. Поэтому если мы хотим, что индексация строк шла от 0 и сами переменные выступали в качестве значений, а не индексов, добавляем в метод **groupby** параметр **as_index=False**:

In [24]:
students_performance.groupby("gender", as_index=False).aggregate({
    "math score": "mean",
    "reading score": "mean"
})

,gender,math score,reading score
0,female,63.633205,72.608108
1,male,68.728216,65.473029


Обозначим, что в наших колонках **math score** и **reading score** хранятся как раз таки средние значения этих параметров:

In [25]:
students_performance.groupby("gender", as_index=False).aggregate({
    "math score": "mean",
    "reading score": "mean"
})\
.rename(columns={
    "math score": "mean_math_score",
    "reading score": "mean_reading_score"
})

,gender,mean_math_score,mean_reading_score
0,female,63.633205,72.608108
1,male,68.728216,65.473029


Также нам ничего не мешает группировать датафреймы сразу по нескольким переменным:

In [26]:
students_performance.groupby(["gender", "race/ethnicity"], as_index=False).aggregate({
    "math score": "mean",
    "reading score": "mean"
})\
.rename(columns={
    "math score": "mean_math_score",
    "reading score": "mean_reading_score"
})

,gender,race/ethnicity,mean_math_score,mean_reading_score
0,female,group A,58.527778,69.000000
1,female,group B,61.403846,71.076923
2,female,group C,62.033333,71.944444
3,female,group D,65.248062,74.046512
4,female,group E,70.811594,75.840580
5,male,group A,63.735849,61.735849
6,male,group B,65.930233,62.848837
7,male,group C,67.611511,65.424460
8,male,group D,69.413534,66.135338
9,male,group E,76.746479,70.295775


А если бы мы убрали параметр **as_index**:

In [28]:
mean_scores = students_performance.groupby(["gender", "race/ethnicity"]).aggregate({
    "math score": "mean",
    "reading score": "mean"
})\
.rename(columns={
    "math score": "mean_math_score",
    "reading score": "mean_reading_score"
})

mean_scores

mean_math_score  mean_reading_score
gender race/ethnicity                                     
female group A               58.527778           69.000000
       group B               61.403846           71.076923
       group C               62.033333           71.944444
       group D               65.248062           74.046512
       group E               70.811594           75.840580
male   group A               63.735849           61.735849
       group B               65.930233           62.848837
       group C               67.611511           65.424460
       group D               69.413534           66.135338
       group E               76.746479           70.295775

Стоит отметить, что такое представление усложняет работу с датафреймами.

Взглянем на характер индексов в данном датафрейме. Получим *мультииндексы*, состоящие из нескольких уровней.
<br>Docs: https://pandas.pydata.org/docs/user_guide/advanced.html

In [29]:
mean_scores.index

MultiIndex([('female', 'group A'),
            ('female', 'group B'),
            ('female', 'group C'),
            ('female', 'group D'),
            ('female', 'group E'),
            (  'male', 'group A'),
            (  'male', 'group B'),
            (  'male', 'group C'),
            (  'male', 'group D'),
            (  'male', 'group E')],
           names=['gender', 'race/ethnicity'])

Данная особенность усложняет обращение к определенным элементам датафрейма. В этом случае нам надо будет обращатся к записям с помощью составного индекса:

In [31]:
mean_scores.loc[("female", "group A")]

mean_math_score       58.527778
mean_reading_score    69.000000
Name: (female, group A), dtype: float64

In [32]:
mean_scores.loc[[("female", "group A"), ("male", "group B")]]

,,mean_math_score,mean_reading_score
gender,race/ethnicity,,
female,group A,58.527778,69.000000
male,group B,65.930233,62.848837


Теперь решим такую задачку: отберем по топ-5 студентов, сгруппированных по переменной **gender**:

In [34]:
students_performance\
    .sort_values(["gender", "math score"], ascending=False)\
    .groupby("gender").head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
149,male,group E,associate's degree,free/reduced,completed,100,100,93
623,male,group A,some college,standard,completed,100,96,86
625,male,group D,some college,standard,completed,100,97,99
916,male,group E,bachelor's degree,standard,completed,100,100,100
306,male,group E,some college,standard,completed,99,87,81
451,female,group E,some college,standard,none,100,92,97
458,female,group E,bachelor's degree,standard,none,100,100,100
962,female,group E,associate's degree,standard,none,100,100,100
114,female,group E,bachelor's degree,standard,completed,99,100,100
263,female,group E,high school,standard,none,99,93,90
